**Notebook To Test Manually The Delay Adder**

1. Import libraries & Initialize simulator:

In [4]:
import pandas as pd
from itertools import product
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

# Initialize simulator
aer_sim = AerSimulator()

2. Adder function:

In [5]:
# Function to create the delay circuit for a given combination of inputs
def create_delay_circuit(t1, delta_t1, t2, delta_t2):
    n_qubits = 11
    qreg = QuantumRegister(n_qubits)
    creg = ClassicalRegister(2, name="c")
    delay_circuit = QuantumCircuit(qreg, creg)

    # Set time switches based on inputs
    if t1:
        delay_circuit.x(5)  # t1
    if delta_t1:
        delay_circuit.x(6)  # delta_t1
    if t2:
        delay_circuit.x(8)  # t2
    if delta_t2:
        delay_circuit.x(9)  # delta_t2

    # Original circuit logic
    triplets = [(9, 8, 7, 10), (6, 5, 4, 7)]
    for ctrl1, ctrl2, target, next_ctrl in triplets:
        delay_circuit.ccx(ctrl1, ctrl2, target)
        delay_circuit.cx(ctrl1, ctrl2)
        delay_circuit.ccx(next_ctrl, ctrl2, target)

    delay_circuit.cx(7, 5)

    qubit_sets = [(10, 8, 7, 9)]
    for ctrl1, ctrl2, target, alt_ctrl in qubit_sets:
        delay_circuit.ccx(ctrl1, ctrl2, target)
        delay_circuit.cx(alt_ctrl, ctrl2)
        delay_circuit.ccx(alt_ctrl, ctrl2, target)
        delay_circuit.cx(alt_ctrl, ctrl2)
        delay_circuit.cx(ctrl1, ctrl2)

    target_pairs = [(2, 3), (3, 2), (0, 1), (1, 0)]
    for ctrl2, target in target_pairs:
        delay_circuit.ccx(4, ctrl2, target)

    # Measurements
    delay_circuit.measure(5, 1)
    delay_circuit.measure(8, 0)

    return delay_circuit

4. Table:

In [6]:
# Create a table to store the results
results = []

# Loop through all 16 combinations of the time switches
for t1, delta_t1, t2, delta_t2 in product([0, 1], repeat=4):
    # Create and compile the circuit for this combination
    circuit = create_delay_circuit(t1, delta_t1, t2, delta_t2)

    # Simulate the circuit
    job = aer_sim.run(circuit, shots=1)
    result = job.result()
    counts = result.get_counts()

    # Add the results to the table
    results.append({
        "t1": t1,
        "t2": t2,
        "delta_t1": delta_t1,
        "delta_t2": delta_t2,
        "counts": counts
    })

# Convert the results to a Pandas DataFrame
results_df = pd.DataFrame(results)

# Display the table to the user
print(results_df.to_markdown())

# How the adder works:
# t1 is more significant than t2, and delta_t1 is more significant than delta_t2.
# The adder takes two 2-bit numbers as input, t1t2 and delta_t1delta_t2, and adds them together.
# The result is stored in the qubits 0 and 1.
# 00 = 0
# 01 = 1
# 10 = 2
# 11 = 3
# 00 + 00 = 00
# 00 + 01 = 01
# ...

|    |   t1 |   t2 |   delta_t1 |   delta_t2 | counts    |
|---:|-----:|-----:|-----------:|-----------:|:----------|
|  0 |    0 |    0 |          0 |          0 | {'00': 1} |
|  1 |    0 |    0 |          0 |          1 | {'01': 1} |
|  2 |    0 |    1 |          0 |          0 | {'01': 1} |
|  3 |    0 |    1 |          0 |          1 | {'10': 1} |
|  4 |    0 |    0 |          1 |          0 | {'10': 1} |
|  5 |    0 |    0 |          1 |          1 | {'11': 1} |
|  6 |    0 |    1 |          1 |          0 | {'11': 1} |
|  7 |    0 |    1 |          1 |          1 | {'00': 1} |
|  8 |    1 |    0 |          0 |          0 | {'10': 1} |
|  9 |    1 |    0 |          0 |          1 | {'11': 1} |
| 10 |    1 |    1 |          0 |          0 | {'11': 1} |
| 11 |    1 |    1 |          0 |          1 | {'00': 1} |
| 12 |    1 |    0 |          1 |          0 | {'00': 1} |
| 13 |    1 |    0 |          1 |          1 | {'01': 1} |
| 14 |    1 |    1 |          1 |          0 | {'01': 1}